# Data Wrangling  


[Column Organization](#organize)
[Numerical](#numerical)
* [Cleaning Numerical Strings](#cleanstring)
    * [Currency](#currency)
    * [Percentage](#percentage)
    * [All](#All)
* [Remove Inactive Listings](#inactive)
    * [Square Feet - Transform](#squarefeet)
    * [Weekly / Monthly Price - Transform](#wmprice)
    * [Bedrooms / Bathrooms / Beds - Imputation](#bbb)
    * [Review Scores - Imputation](#reviewscores)
    * [Security Deposit / Cleaning Fee - Imputation](#sdcf)
    * [Host Response Rate - Remove](#hostresponserate)
    
[Date/Time Columns](#datetime)
* [Calendar Last Updated](#lastupdated)
* [Host Since / First Review / Last Review](#otherdate)

[Open Response Columns](#openresponses)

[Categorical Columns](#categorical)
* [Amenities](#amenities)
* [Categorical - Binary](#binary)
* [Categorical - Multiple Levels](#multiple)
* [Host Response Time](#response)
[Add Amenities to Listings DataFrame](#addamenities)


[Current Listings State](#currentlistings)



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt # plotting library
%matplotlib inline 
# Automatically shows graph after made in Jupyter Notebook 
import seaborn as sns 
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', 100)

<a id='listings'></a>
# Listings

In [2]:
listings = pd.read_csv('listings.csv', low_memory = False)

In [3]:
listings.shape

(8933, 96)

In [4]:
listings.T.iloc[:,:5]

,0,1,2,3,4
id,16570521,8001958,8388658,16064495,15664146
listing_url,https://www.airbnb.com/rooms/16570521,https://www.airbnb.com/rooms/8001958,https://www.airbnb.com/rooms/8388658,https://www.airbnb.com/rooms/16064495,https://www.airbnb.com/rooms/15664146
scrape_id,20171002002039,20171002002039,20171002002039,20171002002039,20171002002039
last_scraped,2017-10-02,2017-10-02,2017-10-02,2017-10-02,2017-10-02
name,Lands' End hideaway,Ocean and park front panoramic view,Mid-century Seacliff near GG Bridge,Charming home with views of the Golden Gate Bridge,Spacious 3 beds in convenient/safe neighborhood
summary,Light and airy in-law with dedicated bathroom and kitchenette. Cozy hideaway in a nice resident...,"This is a classic 1969 building in front of several tourist attractions like the Cliff House, Su...",Three-bedroom in exclusive Seacliff neighborhood in SF; views of the Golden Gate bridge and 1km...,Welcome to one of the coziest and charming homes in Sea Cliff. We are three blocks from Baker Be...,"My home is in one of the safest neighborhoods and walking distance to the golf course, Ocean and..."
space,"The room is on the lower level of the house, with its own bathroom. The room is cozy with an el...",NaN,NaN,This house has lots of history! It was originally the carriage house for the Barbagelata Estate....,NaN
description,Light and airy in-law with dedicated bathroom and kitchenette. Cozy hideaway in a nice resident...,"This is a classic 1969 building in front of several tourist attractions like the Cliff House, Su...",Three-bedroom in exclusive Seacliff neighborhood in SF; views of the Golden Gate bridge and 1km...,Welcome to one of the coziest and charming homes in Sea Cliff. We are three blocks from Baker Be...,"My home is in one of the safest neighborhoods and walking distance to the golf course, Ocean and..."
experiences_offered,none,none,none,none,none
neighborhood_overview,The area around Lincoln Park is a residential part of town and is very peaceful. The Lands End w...,NaN,NaN,Three blocks away on Clement Street are plenty of terrific restaurants.,NaN


#### Delete Unneeded columns

Information we can get rid of:
 * <b> Meta Information:</b> last_scraped, scrape_id, host_id
 * <b>URLs:</b> listing_url, thumbnail_url, medium_url, picture_url, xl_picture_url, host_url, host_thumbnail_url, host_picture_url
 * <b>Redundant Information:</b> city, state, market, smart_location, country_code, country, jurisdiction_names, latitude, longitude, is_location_exact, host_total_listings_count,host_listings_count, neighbourhood, neighbourhood_group_cleansed
 * <b>Irrelevant Information About Host:</b> license, host_location, host_neighbourhood,host_verifications,  host_name, 
 * <b>Bad Information:</b> availability_30, availability_60, availability_90, availability_365, experiences_offered, calendar_last_scraped, has_availability, host_acceptance_rate
 * Any column with <b>1 or less unique values</b>
    

In [5]:
# There are columns that don't provide information we can gain a lot of insights from. We can just remove them
delete_list = ['listing_url','scrape_id','last_scraped','thumbnail_url','medium_url','picture_url',
               'xl_picture_url','host_url','host_thumbnail_url','host_picture_url','city','state','market',
               'smart_location','country_code','country','jurisdiction_names','license','latitude',
               'longitude','is_location_exact','host_location','street', 'host_verifications','requires_license',
               'host_neighbourhood','host_name', 'neighbourhood','availability_30','availability_60',
               'availability_90', 'availability_365','host_listings_count', 'host_total_listings_count','host_id',
              'experiences_offered','host_about','calendar_last_scraped','has_availability','host_acceptance_rate',
              'neighbourhood_group_cleansed']
for column in delete_list:
    del listings[column]

In [6]:
'''Creating a for loop to iterate through the columns and delete all columns that have 1 value or less. 
There is no reason to include these, if all listings have the same value'''
for listing in listings.columns:
    if listings[listing].nunique() <= 1:
        del listings[listing]

In [7]:
# Current rows and columns
listings.shape

(8933, 55)

<a id='organize'></a>
<center><h1>Organizating Columns </h1></center>

I'll start by separating the columns into similar columns. This will help with cleaning the data by grouping the columns that have similar problems.

In [8]:
listing_id = ['id']

currency = ['price','security_deposit','cleaning_fee','extra_people','weekly_price','monthly_price']

percent = ['host_response_rate']

numerical = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included', 'minimum_nights', 
             'maximum_nights' ,'number_of_reviews', 'calculated_host_listings_count', 'reviews_per_month', 
             'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
             'review_scores_communication', 'review_scores_location', 'review_scores_value']


date_time = ['host_since','first_review','last_review','calendar_updated']

open_response = ['name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access',
                 'interaction', 'house_rules']

list_in_column = ['amenities']

categorical_data = ['host_response_time','neighbourhood_cleansed','property_type',
                       'room_type','bed_type','cancellation_policy', 'zipcode']

true_false = ['host_is_superhost','host_has_profile_pic','instant_bookable','is_business_travel_ready',
              'require_guest_profile_picture','require_guest_phone_verification', 'host_identity_verified']

review_scores = ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
                       'review_scores_communication','review_scores_location','review_scores_value']

dependent_variables = ['review_scores_rating','price']

<a id='numerical'></a>

<center><h1> Numerical</h1></center>

We will first work on all numerical columns, but first lets clean a few, so that they are the same as the others

<a id='cleanstring'></a>
## Clean String of Numerical Columns

Seven columns have been seperated into either currency or percentage (6 & 1 respectively) because they were loaded as string with special characters. We keep them seperated to isolate the columns that need the same treatment of cleaning before we can make them of the numerical type.

<a id='currency'></a>
> ### Currency

We seperated this column because the values have dollar signs, and commas in text form. 

In [9]:
for item in currency:
    listings[item] = listings[item].str.replace(',', '')
    listings[item] = listings[item].str.replace('$', '')
    listings[item] = listings[item].astype(float)
    numerical.append(item)

In [10]:
listings[currency].head()

,price,security_deposit,cleaning_fee,extra_people,weekly_price,monthly_price
0,120.0,150.0,35.0,0.0,NaN,NaN
1,1850.0,NaN,NaN,0.0,NaN,NaN
2,895.0,250.0,125.0,0.0,6000.0,NaN
3,975.0,500.0,150.0,0.0,NaN,NaN
4,295.0,500.0,150.0,20.0,NaN,NaN


<a id='percentage'></a>
> ### Percentage

Just as above, this column needs to be changed before it can be turned into a numerical value. This needs the '%' character removed

In [11]:
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '')
listings['host_response_rate'] = listings['host_response_rate'].astype(float)
numerical.append('host_response_rate')

<a id='all'></a>
> ### All Numerical Columns

The remaining numerical columns can be change to numeric easily now

In [12]:
# The CSV read all columns in as strings,
# so we can create a for loop to turn all the columns that should be numerical into numerical values.

for item in numerical:
    listings[item] = pd.to_numeric(listings[item])

<a id='inactive'></a>
## Remove Inactive Listings

Now that our data is in the numerical type, we can find and remove any listings that are no longer active. The reason to get rid of these listing is that our goal is to find ways to increase the ratings and price of listings and if a listing is inactive then they cannot benefit from our model. We should remove them since they are not a good representation of the population we are trying to help, so they may negatively affect our predictive performance. 

Using the column 'number_of_reviews' seems like an intuitive way to find and remove these inactive listings.

In [13]:
len(listings[listings['number_of_reviews'] == 0])

1810

In [14]:
'''Removing any rows without reviews. Not only does this prevent inactive listings from skewing the data,
but also makes a NLP easier for the reviews '''
listings = listings[listings['number_of_reviews'] != 0]
listings = listings[listings['review_scores_rating'].isnull() == False]
listings.shape

(7027, 55)

<a id='numericnull'></a>
## Numerical Null Values

Most predictive models cannot have missing values. If a column has missing values then we need to either remove it, transform it, or impute the missing values.

In [15]:
# Find if any columns have many null values
print('Percent Null (%)')
print((listings[numerical].isnull().sum()/len(listings)*100)[listings[numerical].isnull().sum() > 0])


Percent Null (%)
bathrooms                       0.370001
bedrooms                        0.056923
beds                            0.099616
square_feet                    97.680376
review_scores_accuracy          0.128077
review_scores_cleanliness       0.113847
review_scores_checkin           0.384232
review_scores_communication     0.142308
review_scores_location          0.370001
review_scores_value             0.412694
security_deposit               38.095916
cleaning_fee                   16.265832
weekly_price                   74.313363
monthly_price                  80.162231
host_response_rate             20.321617
dtype: float64


<a id='squarefeet'></a>
> ### Square Feet - Transform

The square feet column has over 97% of its rows missing. Instead of just removing the column, we can transform it to a column that states whether a listings has included square feet. This may have a noteable relationship with reviews or price.

In [16]:
# Instead of removing this column we can make it a true/false column. 
listings['has_square_feet'] = 0
for index, item in listings['square_feet'].iteritems():
    if item > 0:
        listings.loc[index,'has_square_feet'] = True
    else:
        listings.loc[index,'has_square_feet'] = False

In [17]:
listings['has_square_feet'].value_counts()

False    6870
True      157
Name: has_square_feet, dtype: int64

In [18]:
del listings['square_feet']
numerical.remove('square_feet')
true_false.append('has_square_feet')

<a id='wmprice'></a>
> ### Weekly / Monthly Price - Transform

These columns have a high amount of missing values. These columns represent a seperate price (that is a discount) if you are to book by the over a Week / Month. We can just transform these to binary values that say if there is a discount or not for a longer stay

In [19]:
# Instead of removing these column we can make it a true/false column also!
discounts = ['weekly_price','monthly_price']
for discount in discounts:
    listings['has_'+discount] = 0
    for index, item in listings[discount].iteritems():
        if item > 0:
            listings.loc[index, 'has_'+discount] = True
        else:
            listings.loc[index,'has_'+discount] = False

In [20]:
del listings['weekly_price']
del listings['monthly_price']
numerical.remove('weekly_price')
numerical.remove('monthly_price')
true_false.append('has_weekly_price')
true_false.append('has_monthly_price')

<a id='bbb'></a>
> ### Bedrooms / Bathrooms / Beds - Imputation

We can impute the missing values based on with whatever column the missing value column is most correlated with. Then we can  , since we have no missing values for accommodates

In [21]:
listings[['bedrooms','bathrooms','beds','accommodates']].corr()

,bedrooms,bathrooms,beds,accommodates
bedrooms,1.000000,0.477800,0.760590,0.730449
bathrooms,0.477800,1.000000,0.421634,0.398370
beds,0.760590,0.421634,1.000000,0.822428
accommodates,0.730449,0.398370,0.822428,1.000000


In [22]:
print('Null Values: \n ',listings[['bedrooms','bathrooms','beds']].isnull().sum())

Null Values: 
  bedrooms      4
bathrooms    26
beds          7
dtype: int64


Bedrooms and Beds are highly correlation with each other and accommodates. We can use the median of accommodates to impute for both, since accommodates isn't missing any values. 

Bathrooms is most correlated with Bedrooms, so we can do the same since it won't be missing any values after we impute it (Bedrooms) with accommodates.

In [23]:
# Best to impute by median, so we don't get an impossible fraction of rooms to a listings
listings['bedrooms'] = listings.groupby(['accommodates'])['bedrooms'].apply(lambda x: x.fillna(x.median()))
listings['beds'] = listings.groupby(['accommodates'])['beds'].apply(lambda x: x.fillna(x.median()))
listings['bathrooms'] = listings.groupby(['bedrooms'])['bathrooms'].apply(lambda x: x.fillna(x.median()))

<a id='reviewscores'></a>
> ### Review Scores - Imputation

We can impute with Review_scores_rating since no columns are missing

In [24]:
listings[review_scores].corr()

,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
review_scores_rating,1.000000,0.742847,0.721255,0.584882,0.621148,0.471545,0.728607
review_scores_accuracy,0.742847,1.000000,0.633407,0.535994,0.556626,0.377914,0.647665
review_scores_cleanliness,0.721255,0.633407,1.000000,0.474217,0.471555,0.329736,0.592328
review_scores_checkin,0.584882,0.535994,0.474217,1.000000,0.630483,0.304552,0.494428
review_scores_communication,0.621148,0.556626,0.471555,0.630483,1.000000,0.327535,0.502273
review_scores_location,0.471545,0.377914,0.329736,0.304552,0.327535,1.000000,0.448569
review_scores_value,0.728607,0.647665,0.592328,0.494428,0.502273,0.448569,1.000000


For all review scores except for communication and checkin, rating is the most correlated. We should still use rating since it has no missing values, and is still highly correlated. We can bucket review_scores_rating and then impute with the median for all reviews. 

In [25]:
listings['review_scores_accuracy'] = listings.groupby(
    [pd.cut(listings['review_scores_rating'], 20)])['review_scores_accuracy'].apply(
    lambda x: x.fillna(x.median()))

In [26]:
for review_score in review_scores:
    listings[review_score] = listings.groupby(
    [pd.cut(listings['review_scores_rating'], 20)])[review_score].apply(
    lambda x: x.fillna(x.median()))

In [27]:
listings[review_scores].isnull().sum()

review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
dtype: int64

<a id='sdcf'></a>
> ### Security Deposit / Cleaning Fee - Imputation

In [28]:
listings.corr()[['security_deposit','cleaning_fee']].sort_values(by='security_deposit', ascending=False).T

,security_deposit,cleaning_fee,bedrooms,price,accommodates,beds,bathrooms,guests_included,minimum_nights,extra_people,review_scores_rating,review_scores_cleanliness,review_scores_location,review_scores_accuracy,review_scores_checkin,review_scores_communication,review_scores_value,maximum_nights,host_response_rate,number_of_reviews,calculated_host_listings_count,id,reviews_per_month
security_deposit,1.000000,0.469107,0.320587,0.316773,0.270931,0.264898,0.264464,0.185329,0.128960,0.095654,0.082667,0.072108,0.063993,0.057986,0.046212,0.029958,0.022923,-0.008387,-0.025026,-0.092186,-0.093817,-0.136189,-0.204858
cleaning_fee,0.469107,1.000000,0.589978,0.510171,0.561172,0.564413,0.448059,0.371693,0.122545,0.153209,0.090810,0.100404,0.067757,0.056601,0.023357,0.023801,0.011664,-0.002282,-0.025440,-0.156380,-0.020132,-0.108981,-0.233087


<b> Security Deposit</b> is most correlated with cleaning fee(.47), bedrooms(.32), and price(.32).

<b> Cleaning Fee </b> is most correlated with accommodates(.56), bedrooms(.59), beds(.56), price(.51), and security deposit(.47).

We should impute cleaning fee first with bedrooms. Then we can impute the security deposit with the cleaning fee

In [29]:
listings['cleaning_fee'] = listings.groupby(['bedrooms'])['cleaning_fee'].apply(lambda x: x.fillna(x.median()))
listings['security_deposit'] = listings.groupby([pd.cut(listings['cleaning_fee'], 20)])['security_deposit'].apply(lambda x: x.fillna(x.median()))

<a id='hostresponserate'></a>
> ### Host Response Rate - Remove

In [30]:
listings.corr()[['host_response_rate']].sort_values(by='host_response_rate', ascending=False).T

,host_response_rate,reviews_per_month,review_scores_cleanliness,number_of_reviews,review_scores_communication,review_scores_checkin,review_scores_rating,review_scores_accuracy,review_scores_value,calculated_host_listings_count,id,guests_included,extra_people,beds,accommodates,bedrooms,maximum_nights,review_scores_location,price,bathrooms,security_deposit,cleaning_fee,minimum_nights
host_response_rate,1.0,0.135073,0.104765,0.086321,0.074835,0.060123,0.059082,0.056795,0.054445,0.054416,0.045187,0.035872,0.024923,0.020936,0.020229,0.003974,0.003616,-0.000678,-0.006568,-0.006793,-0.009342,-0.015245,-0.017599


In [31]:
listings['host_response_rate'].describe()

count    5599.000000
mean       97.069298
std        10.831516
min         0.000000
25%       100.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: host_response_rate, dtype: float64

No columns have a strong relationship with the host response rate and the majority of its own rows have 100% host response rate. Removing this column may be the best choice, since at first glance, I wouldn't assume it would be a strong predictor.

In [32]:
del listings['host_response_rate']
numerical.remove('host_response_rate')


<a id='datetime'></a>
<center><h1> Date/Time Columns </h1></center>

We need to turn our date columns into a numerical type. We can use the datetime.datetime and datetime.timedelta package for this.

In [33]:
from datetime import datetime, timedelta

In [34]:
# The data was scrapped on October 2, 2017, so all dates are relative to this date
data_date = datetime.strptime('2017/10/02', '%Y/%m/%d')

<a id='lastupdated'></a>
> ### Calendar Last Updated

The calendar_last_updated column uses a wide variety of categorical values that we can be trace back to a specific date. Our reference point is October 2nd, 2017 when all this data was scrapped from AirBnB

In [35]:
# Let's look at calendar_updated first, since it is the farthest from the date format
listings['calendar_updated'].value_counts()[:10]

today           1211
2 weeks ago      551
a week ago       524
yesterday        428
3 weeks ago      402
2 months ago     369
3 days ago       321
4 days ago       309
3 months ago     238
4 weeks ago      235
Name: calendar_updated, dtype: int64

In [36]:
# Changing the strings into usable date format
listings['days_since_calendar_updated'] = 0
for index, item in listings['calendar_updated'].iteritems():
    if item == 'never':
        listings.loc[index,'days_since_calendar_updated'] = -1
    elif item == 'today':
        listings.loc[index,'days_since_calendar_updated'] = 0
    elif item == 'yesterday':
        listings.loc[index,'days_since_calendar_updated'] = 1
    elif item.find('days') != -1:
        listings.loc[index,'days_since_calendar_updated'] = int(item[:2])
    elif item.find('week ago') != -1:
        listings.loc[index,'days_since_calendar_updated'] = 7
    elif item.find('weeks') != -1:
        listings.loc[index,'days_since_calendar_updated'] = int(item[:2])*7
    elif item.find('months') != -1:
        listings.loc[index,'days_since_calendar_updated'] = int(item[:2])*30

In [37]:
# -1 indicated the calendar has never been updated
listings = listings[listings['days_since_calendar_updated'] != -1]
del listings['calendar_updated']
date_time.remove('calendar_updated')
numerical.append('days_since_calendar_updated')

In [38]:
listings['days_since_calendar_updated'].describe()

count    7025.000000
mean       82.591459
std       163.931733
min         0.000000
25%         2.000000
50%        14.000000
75%        60.000000
max      1500.000000
Name: days_since_calendar_updated, dtype: float64

<a id='otherdate'></a>
> ### Host Since / First Review / Last Review

In [39]:
listings[date_time].head()

,host_since,first_review,last_review
0,2014-09-14,2017-02-22,2017-09-04
4,2016-10-24,2016-11-26,2017-09-14
5,2012-05-02,2013-08-28,2017-09-26
6,2013-03-21,2017-07-22,2017-09-20
7,2017-04-02,2017-05-12,2017-09-17


In [40]:
listings.shape

(7025, 54)

In [41]:
listings.dropna(subset=['host_since','first_review','last_review'], inplace=True)

In [42]:
for column in date_time:
    for index, date in listings[column].iteritems():
        listings.loc[index, column] =  datetime.strptime(date, '%Y-%m-%d')

#### Host Since

In [43]:
# Changing from number of days to date format
listings['host_since_days'] = 0
for index, date in listings['host_since'].iteritems():
    listings.loc[index,'host_since_days'] = (data_date - date).days
numerical.append('host_since_days')
del listings['host_since']

#### First Review

In [44]:
# Changing from number of days to date format
listings['first_review_days'] = 0
for index, date in listings['first_review'].iteritems():
    listings.loc[index,'first_review_days'] = (data_date - date).days
numerical.append('first_review_days')
del listings['first_review']

#### Last Review

In [45]:
# Changing from number of days to date format
listings['last_review_days'] = 0
for index, date in listings['last_review'].iteritems():
    listings.loc[index,'last_review_days'] = (data_date - date).days
numerical.append('last_review_days')
del listings['last_review']

In [46]:
listings[['last_review_days','first_review_days','host_since_days','days_since_calendar_updated']].isnull().sum()

last_review_days               0
first_review_days              0
host_since_days                0
days_since_calendar_updated    0
dtype: int64

<a id='openresponses'></a>
<center><h1>Text Columns</h1></center>

Since we can't impute for missing values in open resonse columns, we can just fill missing values with blanks ''. Then we can add the length of all the open response, because there may be a relationship between the effort put into the listing summary, description, etc. with the ratings or price

In [47]:
listings[open_response].T.iloc[:,:5]

,0,4,5,6,7
name,Lands' End hideaway,Spacious 3 beds in convenient/safe neighborhood,Charming Studio Apt at Ocean Beach,Super Fabulous Mansion!,Hardly Strictly Bluegrass Festival blocks away!
summary,Light and airy in-law with dedicated bathroom and kitchenette. Cozy hideaway in a nice resident...,"My home is in one of the safest neighborhoods and walking distance to the golf course, Ocean and...","NON-SMOKING, charming, private, peaceful hideaway two blocks from Ocean Beach and Golden Gate Pa...",Live in a mansion in the Sea Cliff neighborhood of San Francisco! Walking distance to China beac...,20 min. walk to Hardly Strictly Bluegrass Festival at Golden Gate Park. FREE live music of 90+ a...
space,"The room is on the lower level of the house, with its own bathroom. The room is cozy with an el...",NaN,"This NON-SMOKING, quiet and private 1 bedroom studio (500+ sq.ft.) is a charming, peaceful hidea...",This is a very large home in San Francisco's venerable Sea Cliff neighborhood. Many of these est...,"Our apartment is comfortable, peaceful and quiet. If you want to have a restful, rejuvenating vi..."
description,Light and airy in-law with dedicated bathroom and kitchenette. Cozy hideaway in a nice resident...,"My home is in one of the safest neighborhoods and walking distance to the golf course, Ocean and...","NON-SMOKING, charming, private, peaceful hideaway two blocks from Ocean Beach and Golden Gate Pa...",Live in a mansion in the Sea Cliff neighborhood of San Francisco! Walking distance to China beac...,20 min. walk to Hardly Strictly Bluegrass Festival at Golden Gate Park. FREE live music of 90+ a...
neighborhood_overview,The area around Lincoln Park is a residential part of town and is very peaceful. The Lands End w...,NaN,I'm excited for you to discover why I love living where I live. I am a 2-minute walk from the be...,"China beach, Lincoln Park, Legion of Honor, Clement Street","The house is on the very west end of the Outer Richmond District residential neighborhood, close..."
notes,"Work is incomplete in the backyard, so this space will not be available to guests at this time. ...",NaN,STR#26,"Clement Street has some amazing restaurants - Try Fiorello for Italian, Tia Margareta for authen...","You are a 30 minute drive along the Great Highway to San Francisco International Airport, and a ..."
transit,"Parking is abundant, a rarity in San Francisco. The closest bus lines are the 1-California and 3...",There is a shared (side by side) driveway that guests can share with the owner. There are also t...,Public transport by bus to take you anywhere in the city is a 5-minute walk from my home.,"You can park in our garage, or on El Camino Del Mar, in front of the house. Plenty of parking!",San Francisco is a compact city with an extensive public transportation network that makes getti...
access,We welcome you to share our Living Room and Dining Room space. While the bedroom and bathroom ar...,NaN,Guests enter and exit the private studio via my garage. My guests have their own set of keys to ...,NaN,"You have the whole apartment to yourself with a private keyless entry through the garage, access..."
interaction,"We live full time in the home and have a home office, so there will often be someone available t...",NaN,You decide how much socialization or privacy you'd like. I am available to you in person or by p...,I live close by,"I am willing to help in any way you wish, and will also give you the space you want."
house_rules,This is a shoeless house and slippers are provided for your use. No smoking is permitted inside...,Check out is at 11 am unless otherwise agreed upon,Absolutely NO SMOKING or drugs on property - please respect this rule. I do not want my home sme...,Please use the house responsibly and do not move or handle personal belongings. DO NOT USE THE E...,- Check out: 10:00 am - Quiet hours between 10:00 pm-8:00 am. - No shoes in the house please. I...


In [48]:
listings[open_response].isnull().sum()

name                        4
summary                   273
space                    1521
description                 3
neighborhood_overview    2325
notes                    3317
transit                  2116
access                   2274
interaction              2408
house_rules              1787
dtype: int64

In [49]:
# There are only 3 rows without a name and 2 with no description. I'll just delete these rows.
listings = listings[listings['name'].isnull() == False]
listings = listings[listings['description'].isnull() == False]

In [50]:
# Fill remaining null values as empty strings
for item in open_response:
    listings[item] = listings[item].fillna(' ')

In [51]:
# Create new columns that are the length of the open responses. Effort may have a relationship with reviews
for item in open_response:
    for index, row in listings[item].iteritems():
        if type(row) == str:
            listings.loc[index,item+'_len'] = len(row)
        else:
            listings.loc[index,item+'_len'] = 0

I believe we can combine most of these columns into the average of them all. This way we can measure the effort into the post a little easier. Once we obtain the average length of the text responses, we can delete all the other length columns, except  **house_rules_len**. The reasons why we should keep this one is because if the list of rules is too long, then that may have a relationship with lower reviews. No one wants to have to worry the whole time about a bunch of ruless while on vacation.

In [53]:
listings['average_len'] = listings[['name_len','summary_len','house_rules_len','space_len', 'description_len',
                                     'neighborhood_overview_len','notes_len', 'transit_len',
                                     'access_len', 'interaction_len']].mean(axis=1)

listings.drop(['name_len','summary_len','space_len', 'description_len','neighborhood_overview_len','notes_len',
               'transit_len','access_len', 'interaction_len'], axis=1, inplace=True)

listings.drop(open_response, axis=1,inplace=True)

<a id='categorical'></a>
<center><h1>Categorical Columns</center></h1>

Our categorical data contains Binary columns, Multi-level columns, and a list within each row. We need to turn our categorical data from text into the categorical or boolean type.

<a id='binary'></a>
> ### Categorical - Binary

In [54]:
listings[true_false].head()

,host_is_superhost,host_has_profile_pic,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,host_identity_verified,has_square_feet,has_weekly_price,has_monthly_price
0,f,t,f,f,f,f,t,False,False,False
4,f,t,t,t,f,f,f,False,False,False
5,t,t,t,t,f,f,t,False,True,False
6,f,t,f,f,f,f,f,False,False,False
7,f,t,t,t,f,f,t,False,False,False


In [55]:
for column in true_false:
    for index, item in listings[column].iteritems():
        if item == 't':
            listings.loc[index,column] = True
        elif item == 'f':
            listings.loc[index,column] = False
    print(column.upper())
    print('Value Counts: ','\n',listings[column].value_counts())
    print('Null Values:',listings[column].isnull().sum())
    print('\n')

HOST_IS_SUPERHOST
Value Counts:  
 False    5112
True     1905
Name: host_is_superhost, dtype: int64
Null Values: 0


HOST_HAS_PROFILE_PIC
Value Counts:  
 True     6999
False      18
Name: host_has_profile_pic, dtype: int64
Null Values: 0


INSTANT_BOOKABLE
Value Counts:  
 False    5206
True     1811
Name: instant_bookable, dtype: int64
Null Values: 0


IS_BUSINESS_TRAVEL_READY
Value Counts:  
 False    5752
True     1265
Name: is_business_travel_ready, dtype: int64
Null Values: 0


REQUIRE_GUEST_PROFILE_PICTURE
Value Counts:  
 False    6699
True      318
Name: require_guest_profile_picture, dtype: int64
Null Values: 0


REQUIRE_GUEST_PHONE_VERIFICATION
Value Counts:  
 False    6616
True      401
Name: require_guest_phone_verification, dtype: int64
Null Values: 0


HOST_IDENTITY_VERIFIED
Value Counts:  
 True     5293
False    1724
Name: host_identity_verified, dtype: int64
Null Values: 0


HAS_SQUARE_FEET
Value Counts:  
 False    6861
True      156
Name: has_square_feet, dtype: i

In [56]:
listings[true_false] = listings[true_false].astype('bool')

<a id='multiple'></a>
> ### Categorical - Multiple Levels

In [57]:
listings[categorical_data].head()

,host_response_time,neighbourhood_cleansed,property_type,room_type,bed_type,cancellation_policy,zipcode
0,within an hour,Seacliff,House,Private room,Real Bed,moderate,94121
4,within an hour,Seacliff,Apartment,Entire home/apt,Real Bed,strict,94121
5,within an hour,Seacliff,House,Entire home/apt,Real Bed,flexible,94121
6,within a few hours,Seacliff,House,Entire home/apt,Real Bed,moderate,94121
7,within an hour,Seacliff,Apartment,Entire home/apt,Real Bed,moderate,94121


In [58]:
for column in categorical_data:
    listings[column] = pd.Categorical(listings[column])
    print(column.upper())
    print('Unique Values: ',listings[column].nunique())
    print('Null Values:',listings[column].isnull().sum())
    print('\n')

HOST_RESPONSE_TIME
Unique Values:  4
Null Values: 1421


NEIGHBOURHOOD_CLEANSED
Unique Values:  37
Null Values: 0


PROPERTY_TYPE
Unique Values:  25
Null Values: 0


ROOM_TYPE
Unique Values:  3
Null Values: 0


BED_TYPE
Unique Values:  5
Null Values: 0


CANCELLATION_POLICY
Unique Values:  5
Null Values: 0


ZIPCODE
Unique Values:  30
Null Values: 79




<a id='response'></a>
### Host Response Time



In [59]:
listings['host_response_time'].value_counts()

within an hour        3344
within a few hours    1408
within a day           787
a few days or more      57
Name: host_response_time, dtype: int64

In [60]:
listings.groupby('host_response_time').mean()

,id,host_is_superhost,host_has_profile_pic,host_identity_verified,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,has_square_feet,has_weekly_price,has_monthly_price,days_since_calendar_updated,host_since_days,first_review_days,last_review_days,house_rules_len,average_len
host_response_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a few days or more,6.086104e+06,0.017544,0.982456,0.649123,3.315789,1.289474,1.456140,1.719298,266.736842,422.421053,93.333333,1.754386,16.140351,6.315789,573.456140,19.929825,93.228070,9.561404,9.175439,9.736842,9.631579,9.561404,9.175439,0.087719,0.017544,0.052632,0.035088,1.105263,0.550526,0.035088,0.350877,0.245614,141.280702,1556.982456,982.561404,355.385965,188.684211,232.917544
within a day,8.360155e+06,0.227446,0.996188,0.729352,3.343075,1.285896,1.369759,1.785260,233.569250,369.337992,83.635324,1.696315,15.320203,5.232529,666.898348,28.439644,95.470140,9.742058,9.555273,9.841169,9.814485,9.662008,9.459975,0.049555,0.152478,0.053367,0.123253,3.490470,1.098615,0.021601,0.302414,0.237611,49.304956,1515.235070,802.091487,149.743329,226.706480,265.718043
within a few hours,8.877448e+06,0.318892,0.997869,0.779119,3.229403,1.356534,1.406250,1.771307,215.064631,385.455966,85.458097,1.779119,17.339489,6.132102,1324.272017,36.730114,94.908381,9.718750,9.555398,9.834517,9.834517,9.556108,9.412642,0.097301,0.220881,0.050426,0.058239,2.177557,1.592869,0.026989,0.304688,0.262074,24.794034,1468.904830,773.099432,98.430398,245.174716,289.786009
within an hour,1.091837e+07,0.359151,0.997907,0.744617,3.163577,1.213517,1.262560,1.715012,195.737141,300.399522,72.631728,1.761364,19.594797,3.942584,642679.455144,50.320275,95.024222,9.725778,9.598684,9.854964,9.850778,9.555323,9.423744,0.419258,0.221591,0.054127,0.057416,2.613935,2.672404,0.021531,0.215610,0.185108,16.509868,1298.982656,630.373804,51.172249,279.623804,312.143900


In [61]:
listings['host_response_time'].fillna(method='backfill', inplace=True)

### Zipcode

In [62]:
listings['zipcode'].isnull().sum()

79

In [63]:
zip_dict = dict(listings.groupby('neighbourhood_cleansed')['zipcode'].agg(lambda x:x.value_counts().index[0]))

In [64]:
for index, row in listings[listings['zipcode'].isnull()].iterrows():
    for neighbourhood in zip_dict.keys():
        if row['neighbourhood_cleansed'] == neighbourhood:
            listings.loc[index,'zipcode'] = zip_dict[neighbourhood]


In [65]:
listings.loc[listings['zipcode'] == '95202\n\n\n94158', ['zipcode']] = '94158'

In [66]:
listings.loc[listings['zipcode'] == '95202\n\n\n94158', ['zipcode']]

,zipcode


<a id='amenities'></a>
> ### Amenities

The amenities column contains a list of items in each row. We need to clean the text and create dummy variables.

In [67]:
listings[list_in_column].head()

,amenities
0,"{TV,""Wireless Internet"",""Indoor fireplace"",Heating,""Smoke detector"",""Carbon monoxide detector"",""..."
4,"{TV,""Cable TV"",Internet,""Wireless Internet"",Kitchen,""Free parking on premises"",""Indoor fireplace..."
5,"{TV,""Cable TV"",Internet,""Wireless Internet"",""Free parking on premises"",Breakfast,Heating,""Family..."
6,"{TV,""Cable TV"",Kitchen,""Free parking on premises"",Heating,Washer,Dryer,Essentials,Shampoo,""Lapto..."
7,"{TV,""Cable TV"",""Wireless Internet"",Kitchen,""Free parking on premises"",Breakfast,Heating,Washer,D..."


In [68]:
listings['amenities'] = listings['amenities'].str.replace('}',"")
listings['amenities'] = listings['amenities'].str.replace('{',"")
listings['amenities'] = listings['amenities'].str.replace('"',"")
listings['amenities'] = listings['amenities'].str.split(',')


In [69]:
listings['amenities'].apply(pd.Series).stack().nunique()

121

In [70]:
pd.DataFrame(listings['amenities'].apply(pd.Series).stack().value_counts())[:10]

,0
Wireless Internet,6866
Heating,6454
Essentials,6276
Smoke detector,6144
Kitchen,6054
Shampoo,5329
TV,5113
Hangers,4967
Carbon monoxide detector,4951
Hair dryer,4807


In [71]:
amenities = pd.concat([pd.get_dummies(listings['amenities'].apply(pd.Series).stack()).sum(level=0), 
        listings[['id']]], axis=1)

In [72]:
amenities.drop(['id'],axis=1).apply(pd.Series.value_counts).T.sort_values(by=1.0, ascending=False).T

,Wireless Internet,Heating,Essentials,Smoke detector,Kitchen,Shampoo,TV,Hangers,Carbon monoxide detector,Hair dryer,Washer,Laptop friendly workspace,Dryer,Iron,Internet,Fire extinguisher,Family/kid friendly,First aid kit,Cable TV,translation missing: en.hosting_amenity_50,24-hour check-in,translation missing: en.hosting_amenity_49,Self Check-In,Lock on bedroom door,Free parking on premises,Safety card,Buzzer/wireless intercom,Indoor fireplace,Private entrance,Pets live on this property,Lockbox,Refrigerator,Elevator,Dishes and silverware,Microwave,Bed linens,Hot water,Coffee maker,Cooking basics,Stove,Oven,Breakfast,Air conditioning,Extra pillows and blankets,Dog(s),Pets allowed,Dishwasher,Bathtub,Keypad,Gym,Garden or backyard,Patio or balcony,Luggage dropoff allowed,Hot tub,Cat(s),Wheelchair accessible,Suitable for events,Children’s books and toys,Room-darkening shades,Private living room,Pack ’n Play/travel crib,Long term stays allowed,Smart lock,BBQ grill,Doorman,Well-lit path to entrance,Children’s dinnerware,Smoking allowed,High chair,Single level home,Host greets you,Ethernet connection,Pool,Babysitter recommendations,Stair gates,Wide hallway clearance,smooth pathway to front door,Flat,Outlet covers,Crib,Wide clearance to bed,Accessible-height bed,Game console,Wide entryway,Fireplace guards,Changing table,Step-free access,Baby bath,Wide doorway,Accessible-height toilet,Handheld shower head,Cleaning before checkout,Wide clearance to shower & toilet,Pocket wifi,Hot water kettle,Window guards,Other pet(s),Other,Private bathroom,Firm mattress,Beach essentials,Baby monitor,,Table corner guards,EV charger,Disabled parking spot,Fixed grab bars for shower & toilet,Waterfront,Beachfront,Ground floor access,Bathtub with shower chair,Washer / Dryer,Lake access,Free parking on street,Roll-in shower with chair,Paid parking off premises,Body soap,Bath towel,Toilet paper,Hand or paper towel,Hand soap
0,151.0,563.0,741.0,873.0,963.0,1688.0,1914.0,2050.0,2066.0,2210.0,2386.0,2403.0,2406.0,2503.0,2657.0,2884.0,3500.0,3792.0,4349.0,4575.0,4749.0,5017.0,5088.0,5439.0,5464.0,5516.0,5750.0,5769.0,5829.0,5859.0,5951.0,5986.0,6014.0,6037.0,6087.0,6107.0,6110.0,6145.0,6163.0,6200.0,6221.0,6228.0,6256.0,6286.0,6310.0,6332.0,6372.0,6388.0,6418.0,6535.0,6563.0,6565.0,6582.0,6606.0,6632.0,6688.0,6693.0,6715.0,6721.0,6726.0,6736.0,6758.0,6761.0,6778.0,6771.0,6830.0,6843.0,6856.0,6858.0,6859.0,6868.0,6871.0,6894.0,6897.0,6911.0,6913.0,6916.0,6916.0,6920.0,6926.0,6936.0,6937.0,6940.0,6945.0,6953.0,6958.0,6832.0,6963.0,6895.0,6965.0,6968.0,6981.0,6982.0,6983.0,6984.0,6985.0,6985.0,6989.0,6990.0,6990.0,6990.0,6991.0,6992.0,6995.0,7000.0,7004.0,7006.0,7006.0,7010.0,7011.0,7013.0,7013.0,7013.0,7014.0,7015.0,7015.0,7016.0,7016.0,7016.0,7016.0,7016.0
1,6866.0,6454.0,6276.0,6144.0,6054.0,5329.0,5093.0,4967.0,4951.0,4807.0,4631.0,4614.0,4611.0,4514.0,4360.0,4133.0,3517.0,3225.0,2668.0,2442.0,2268.0,2000.0,1929.0,1578.0,1553.0,1501.0,1267.0,1248.0,1188.0,1158.0,1066.0,1031.0,1003.0,980.0,930.0,910.0,907.0,872.0,854.0,817.0,796.0,789.0,761.0,731.0,707.0,685.0,645.0,629.0,599.0,482.0,454.0,452.0,435.0,411.0,385.0,329.0,324.0,302.0,296.0,291.0,281.0,259.0,256.0,239.0,210.0,187.0,174.0,161.0,159.0,158.0,149.0,146.0,123.0,120.0,106.0,104.0,101.0,101.0,97.0,91.0,81.0,80.0,77.0,72.0,64.0,59.0,55.0,54.0,53.0,52.0,49.0,36.0,35.0,34.0,33.0,32.0,32.0,28.0,27.0,27.0,27.0,26.0,25.0,22.0,17.0,13.0,11.0,11.0,7.0,6.0,4.0,4.0,4.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [73]:
for column in amenities.drop(['id'],axis=1):
    amenities.loc[amenities[column] > 1, column] = 1

Amenities we can drop:
* **Washer / Dryer**: It is redundant information. All those who included it, also selected Washer and Dryer seperate
* **Other**: Doesn't help our analysis, since it could mean anything, also very low frequency
* **Body Soap/Hand Soap/Bath towel/Toilet paper/Hand or paper towel**:
* **Free parking on street**: Very low frequency and not really an amenity


Amenities Groups: There are a few groups of amenities that can be seen in the list of amenities. I think it may help our model if we aggregate two of these groups that stand out:
* **Baby/Child Care Amenities:** There are 13 amenities that specifically cater to adults traveling with kids. By aggregating the total amount of child care supplies provided and advertised by the host, we may find that the amount has a relationship with price or ratings
* **Handicap Accessible Amenities:** There are 18 amenities that can be aggreated that are specifically supplied and advertised to assist those with hanicaps.




In [74]:
amenities = amenities.drop(['Washer / Dryer', 'Other','Hand soap','Bath towel','Toilet paper',
                            'Hand or paper towel','Free parking on street','Body soap'], axis=1)

child_care = ['Table corner guards','Baby monitor','Baby bath','Changing table','Crib','Outlet covers','Stair gates',
              'Babysitter recommendations','High chair','Children’s dinnerware','Pack ’n Play/travel crib','Children’s books and toys',
              'Family/kid friendly']



handicap = ['Roll-in shower with chair','Bathtub with shower chair','Fixed grab bars for shower & toilet',
            'Disabled parking spot','Wide clearance to shower & toilet','Accessible-height toilet','Accessible-height bed',
            'Wide clearance to bed','Wide hallway clearance','Wide entryway',' smooth pathway to front door','Wide doorway',
            'Single level home','Step-free access','Wheelchair accessible','Ground floor access','Single level home','Elevator']


In [75]:
listings = pd.merge(left=listings,right=amenities, left_on='id', right_on='id')
listings.reset_index(drop = True, inplace = True)

In [76]:
listings['total_child_care_amenities'] = listings[child_care].T.sum()

listings['total_handicap_amenities'] = listings[handicap].T.sum()

# Multicollinearity

In [77]:
features = ['price','review_scores_rating','accommodates', 'bathrooms', 'bedrooms', 'beds', 'guests_included', 'minimum_nights', 'maximum_nights',
            'number_of_reviews', 'calculated_host_listings_count', 'reviews_per_month',
            'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
            'review_scores_location', 'review_scores_value','security_deposit','cleaning_fee',
            'extra_people','days_since_calendar_updated', 'average_len', 'house_rules_len', 'host_since_days',
            'first_review_days', 'last_review_days','host_response_time', 'neighbourhood_cleansed',
            'property_type', 'room_type', 'bed_type','cancellation_policy', 'host_is_superhost', 'host_has_profile_pic',
            'instant_bookable', 'is_business_travel_ready','require_guest_profile_picture','require_guest_phone_verification',
            'has_square_feet','zipcode',' smooth pathway to front door','24-hour check-in','Accessible-height bed','Accessible-height toilet',
            'Air conditioning','BBQ grill','Baby bath','Baby monitor','Babysitter recommendations',
            'Bathtub','Bathtub with shower chair','Beach essentials','Beachfront','Bed linens','Breakfast',
            'Buzzer/wireless intercom','Cable TV','Carbon monoxide detector','Cat(s)','Changing table',
            'Children’s books and toys','Children’s dinnerware','Cleaning before checkout','Coffee maker','Cooking basics',
            'Crib','Disabled parking spot','Dishes and silverware','Dishwasher','Dog(s)','Doorman','Dryer','EV charger',
            'Elevator','Essentials','Ethernet connection','Extra pillows and blankets','Family/kid friendly','Fire extinguisher',
            'Fireplace guards','Firm mattress','First aid kit','Fixed grab bars for shower & toilet','Flat',
            'Free parking on premises','Game console','Garden or backyard','Ground floor access',
            'Gym','Hair dryer','Handheld shower head','Hangers','Heating','High chair',
            'Host greets you','Hot tub','Hot water','Hot water kettle','Indoor fireplace','Internet','Iron','Keypad','Kitchen',
            'Lake access','Laptop friendly workspace','Lock on bedroom door','Lockbox','Long term stays allowed',
            'Luggage dropoff allowed','Microwave','Other pet(s)','Outlet covers','Oven','Pack ’n Play/travel crib',
            'Paid parking off premises','Patio or balcony','Pets allowed','Pets live on this property','Pocket wifi','Pool',
            'Private bathroom','Private entrance','Private living room','Refrigerator','Roll-in shower with chair',
            'Room-darkening shades','Safety card','Self Check-In','Shampoo','Single level home','Smart lock','Smoke detector',
            'Smoking allowed','Stair gates','Step-free access','Stove','Suitable for events','TV','Table corner guards',
            'Washer','Waterfront','Well-lit path to entrance','Wheelchair accessible',
            'Wide clearance to bed','Wide clearance to shower & toilet','Wide doorway','Wide entryway',
            'Wide hallway clearance','Window guards','Wireless Internet']

In [78]:
len(features)

151

In [79]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

def calculate_vif_(X, thresh=10.0):
    variables = list(X.columns)
    removed_variables = []
    dropped=True
    while dropped:
        dropped=False
        vif = [variance_inflation_factor(exog=X[variables].values, exog_idx=ix) for ix,variable in enumerate(list(X[variables].columns))]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('Feature: \'' + X[variables].columns[maxloc] + '\' at index: ' + str(maxloc) + ' with a VIF: ' + str(max(vif)))
            removed_variables.append(X[variables].columns[maxloc])
            del variables[maxloc]
            dropped=True
    print('High VIF Variables: '+str(removed_variables))
    return X[variables]

In [80]:
# Creat new Dataframe of only predictor features
df = listings[features]
# Create dummy variables for categorical data
df = pd.get_dummies(df, columns=categorical_data, drop_first=True)

df = df.astype(np.float64)

# Remove our Response variables
X = df.drop(['price', 'review_scores_rating'], axis=1)


In [81]:
# Calculate variance inflcation factor and remove multicollinear features 
vif_price = calculate_vif_(X, thresh=5)

C:\Users\nolan\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\nolan\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1388: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


Feature: ' smooth pathway to front door' at index: 32 with a VIF: inf
Feature: 'neighbourhood_cleansed_Treasure Island/YBI' at index: 175 with a VIF: inf
Feature: 'review_scores_communication' at index: 13 with a VIF: 620.018433057
Feature: 'review_scores_checkin' at index: 12 with a VIF: 489.284604176
Feature: 'review_scores_accuracy' at index: 10 with a VIF: 429.205822487
Feature: 'host_has_profile_pic' at index: 23 with a VIF: 379.208125451
Feature: 'review_scores_location' at index: 11 with a VIF: 295.907002957
Feature: 'bed_type_Real Bed' at index: 203 with a VIF: 275.750837999
Feature: 'review_scores_value' at index: 11 with a VIF: 230.36813557
Feature: 'review_scores_cleanliness' at index: 10 with a VIF: 136.140536198
Feature: 'zipcode_94110' at index: 213 with a VIF: 131.914245238
Feature: 'host_response_time_within an hour' at index: 136 with a VIF: 67.3188748745
Feature: 'Wireless Internet' at index: 133 with a VIF: 60.0684335183
Feature: 'Washer' at index: 123 with a VIF: 47

### Results:

List of variable with a VIF over 5:

High VIF Variables: \[' smooth pathway to front door', 'neighbourhood_cleansed_Treasure Island/YBI', 'review_scores_communication', 'review_scores_checkin', 'review_scores_accuracy', 'host_has_profile_pic', 'review_scores_location', 'bed_type_Real Bed', 'review_scores_value', 'review_scores_cleanliness', 'zipcode_94110', 'host_response_time_within an hour', 'Wireless Internet', 'Washer', 'zipcode_94121', 'Refrigerator', 'neighbourhood_cleansed_Western Addition', 'accommodates', 'Stove', 'Self Check-In', 'Dishes and silverware', 'Heating', 'Smoke detector', 'Essentials', 'zipcode_94122', 'bedrooms', 'zipcode_94118', 'Cooking basics', 'average_len', 'zipcode_94123', 'Kitchen', 'bathrooms', 'first_review_days', 'Hair dryer', 'property_type_Camper/RV', 'zipcode_94114', 'beds', 'Hangers', 'host_since_days', 'Oven', 'zipcode_94107', 'cleaning_fee', 'Bed linens', 'Shampoo', 'zipcode_94112', 'Microwave', 'zipcode_94133'\]

<a id='currentlistings'></a>
<center><h1>Current State of Listings</center></h1>

In [82]:
listings.shape

(7017, 161)

In [83]:
listings.isnull().sum()[listings.isnull().sum() > 0]

Series([], dtype: int64)

In [84]:
listings.to_csv('listings_clean.csv', encoding='utf-8')